# Transformer for Tokenized Java Processing

source: http://leclair.tech/data/funcom/#procdata

In [26]:
!python preprocess.py \
    -train_src data/funcom/train/functions.train \
    -train_tgt data/funcom/train/comments.train \
    -valid_src data/funcom/valid/functions.valid \
    -valid_tgt data/funcom/valid/comments.valid \
    -shard_size 100000 \
    -save_data data/funcom/_prepped/prepped

[2019-06-05 03:55:44,050 INFO] Extracting features...
[2019-06-05 03:55:44,050 INFO]  * number of source features: 0.
[2019-06-05 03:55:44,050 INFO]  * number of target features: 0.
[2019-06-05 03:55:44,050 INFO] Building `Fields` object...
[2019-06-05 03:55:44,050 INFO] Building & saving training data...
[2019-06-05 03:55:44,050 INFO] Reading source and target files: data/funcom/train/functions.train data/funcom/train/comments.train.
[2019-06-05 03:55:44,097 INFO] Building shard 0.
[2019-06-05 03:55:47,867 INFO]  * saving 0th train data shard to data/funcom/_prepped/prepped.train.0.pt.
[2019-06-05 03:55:50,841 INFO] Building shard 1.
[2019-06-05 03:55:54,421 INFO]  * saving 1th train data shard to data/funcom/_prepped/prepped.train.1.pt.
[2019-06-05 03:55:57,420 INFO] Building shard 2.
[2019-06-05 03:56:00,806 INFO]  * saving 2th train data shard to data/funcom/_prepped/prepped.train.2.pt.
[2019-06-05 03:56:03,455 INFO] Building shard 3.
[2019-06-05 03:56:06,716 INFO]  * saving 3th tr

In [30]:
# transformer
!python train.py \
    -data data/funcom/_prepped/prepped \
    -save_model model/funcom_06042019 \
    -keep_checkpoint 1 \
    -layers 6 \
    -rnn_size 512 \
    -word_vec_size 512 \
    -transformer_ff 2048 \
    -heads 8 \
    -encoder_type transformer \
    -decoder_type transformer \
    -position_encoding \
    -train_steps 200000  \
    -max_generator_batches 0 \
    -dropout 0.1 \
    -batch_size 40 \
    -batch_type tokens \
    -normalization tokens \
    -accum_count 2 \
    -optim adam \
    -adam_beta2 0.998 \
    -decay_method noam \
    -warmup_steps 8000 \
    -learning_rate 0.2 \
    -max_grad_norm 0 \
    -param_init 0  \
    -param_init_glorot \
    -label_smoothing 0.1 \
    -valid_steps 50000 \
    -save_checkpoint_steps 10000 \
    -world_size 1 \
    -gpu_ranks 0

# originally used a learning rate of 2 despite OpenNMT's recommendation 
# of using 0.0001

[2019-06-05 04:11:20,361 INFO]  * src vocab size = 50002
[2019-06-05 04:11:20,362 INFO]  * tgt vocab size = 50004
[2019-06-05 04:11:20,362 INFO] Building model...
[2019-06-05 04:11:25,420 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(50002, 512, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.1)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_keys): Linear(in_features=512, out_features=512, bias=True)
          (linear_values): Linear(in_features=512, out_features=512, bias=True)
          (linear_query): Linear(in_features=512, out_features=512, bias=True)
          (softmax): Softmax()
          (dropout): Dropout(p=0.1)
          (final_linear): Linear(in_features=512, out_features=512, bias=True)
        )
     

In [31]:
!python translate.py \
    -model model/funcom_06042019_step_200000.pt \
    -src data/funcom/test/functions.test \
    -tgt data/funcom/test/comments.test \
    -output pred.txt \
    -report_time \
    -gpu 0

[2019-06-05 17:34:55,409 INFO] Translating shard 0.
PRED AVG SCORE: -1.2553, PRED PPL: 3.5090
GOLD AVG SCORE: -3.6657, GOLD PPL: 39.0816
Total translation time (s): 226.015424
Average translation time (s): 0.022602
Tokens per second: 236.098931
[2019-06-05 17:38:41,691 INFO] Translating shard 1.
PRED AVG SCORE: -1.2895, PRED PPL: 3.6310
GOLD AVG SCORE: -3.8377, GOLD PPL: 46.4206
Total translation time (s): 221.970803
Average translation time (s): 0.022197
Tokens per second: 238.734101
[2019-06-05 17:42:23,875 INFO] Translating shard 2.
PRED AVG SCORE: -1.2815, PRED PPL: 3.6020
GOLD AVG SCORE: -3.8054, GOLD PPL: 44.9431
Total translation time (s): 222.623388
Average translation time (s): 0.022262
Tokens per second: 234.809112
[2019-06-05 17:46:06,729 INFO] Translating shard 3.
PRED AVG SCORE: -1.0835, PRED PPL: 2.9551
GOLD AVG SCORE: -3.6356, GOLD PPL: 37.9252
Total translation time (s): 195.466068
Average translation time (s): 0.019547
Tokens per second: 277.459923
[2019-06-05 17:49:22